# 周报数据生成器

### 使用说明
<br>
整个Code分成两个完全不同的模块：<br> 
    I.  第二部分简报;<br>
    II. 第六部分海信&TCL不同尺寸产品BPI表格。 <br>
    
### I. 第二部分简报
1. 原文件（输入文件）需要改成 weekly market changes.xlsx <br>
2. Excel中的Tab需要改成 Sheet1, 500+ 和 1000+ <br>
3. 表格的格式不需要做任何改动，包括开头的免责声明。 <br>
4. 最后生成为 Part II Chart.xlsx 文件 <br>

### II. 第六部分海信&TCL BPI表格
1. 原文件（输入文件）需要改成 weekly market changes, BPI.xlsx <br>
2. 表格的格式不需要做任何改动，包括开头的免责声明。 <br>
3. 最后生成为 Part VI Chart.xlsx 文件 <br>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [31]:

def load_and_preprocess_data(sheet_name):
    # load excel file and select specific sheet
    df = pd.read_excel('weekly market changes.xlsx', sheet_name)
    # cleanup dataframe
    
    if sheet_name == 'Sheet1': 
        df.columns = df.loc[1]
        df = df.drop(df.index[0:2])
        df = df.replace({'\$': '', ',': ''}, regex=True)
        df = df.reset_index()
        df = df.drop('index', axis = 1)
        
    else:
        df = df.dropna(axis=0,how = 'all')
        df = df.dropna(axis=1,how = 'all')
        df = df.reset_index()
        df = df.drop('index', axis = 1)
        df.columns = df.loc[0]
        df = df.drop(df.index[0])
        df = df.sort_values('Brands')

    return df

def calculate_ratios(use_df):
    relevant_cols = use_df.columns
    last_week_ratios = []
    last_year_ratios = []
    curr_week_ratios = []

    for i in range(0, int(len(relevant_cols)/2), 6):
        last_week_ratio, last_year_ratio, curr_week = compute_ratio(use_df, i)
        last_week_ratios.append(last_week_ratio.tolist())
        last_year_ratios.append(last_year_ratio.tolist())

    for i in range(int(len(relevant_cols)/2), len(relevant_cols), 6):
        last_week_ratio, last_year_ratio, curr_week = minus_ratio(use_df, i)
        last_week_ratios.append(last_week_ratio.tolist())
        last_year_ratios.append(last_year_ratio.tolist())
        curr_week_ratios.append(curr_week.round(1).tolist())
    return last_week_ratios, last_year_ratios, curr_week_ratios

def compute_ratio(use_df, i):
    curr_week = use_df.iloc[:, i+3].astype(float)
    last_week = use_df.iloc[:, i+2].astype(float)
    last_year = use_df.iloc[:, i+1].astype(float)
    last_week_ratio = ((curr_week - last_week) / last_week * 100).round(1)
    last_year_ratio = ((curr_week - last_year) / last_year * 100).round(1)
    return last_week_ratio, last_year_ratio, curr_week

def minus_ratio(use_df, i):
    curr_week = use_df.iloc[:, i+3].astype(float)
    last_week = use_df.iloc[:, i+2].astype(float)
    last_year = use_df.iloc[:, i+1].astype(float)

    last_week_ratio = (curr_week - last_week).round(1)
    last_year_ratio = (curr_week - last_year).round(1)
    return last_week_ratio, last_year_ratio, curr_week

def format_ratio(ratio_list):
    return [f'+{n}' if n > 0 else str(n) for n in ratio_list]

def get_week_number(date_str):
    second_date_str = date_str.split('-')[1].strip()
    second_date = datetime.strptime(second_date_str, '%b %d %Y')
    days_since_start_of_year = (second_date - datetime(second_date.year, 1, 1)).days + 1
    return days_since_start_of_year // 7

In [32]:
# load data
data = load_and_preprocess_data('Sheet1')
brand_list = ['Brand','Grand Total','Hisense','LG','Samsung','Sony','TCL','VIZIO']
data = data[data.iloc[:,0].isin(brand_list)]
data2 = load_and_preprocess_data('500+')
data3 = load_and_preprocess_data('1000+')

# calculate share
fivehun_dollar_share = (data2['Dollar Share'].astype(float) *100).round(1).tolist()
fivehun_dslw = ((data2['Dollar Share'].astype(float) - data2['Dollar Share LW'].astype(float))*100).round(1).tolist()
fivehun_dsly = ((data2['Dollar Share'].astype(float) - data2['Dollars Share LY'].astype(float))*100).round(1).tolist()

thou_dollar_share = (data3['Dollar Share'].astype(float) *100).round(1).tolist()
thou_dslw = ((data3['Dollar Share'].astype(float) - data3['Dollar Share LW'].astype(float))*100).round(1).tolist()
thou_dsly = ((data3['Dollar Share'].astype(float) - data3['Dollars Share LY'].astype(float))*100).round(1).tolist()
# get week number
if '(Dec' in (data.iloc[0,6][:13].split()): 
    current_nweek = get_week_number(data.iloc[0,4])+1
else:
    current_nweek = get_week_number(data.iloc[0,4])

# compute ratios
use_df = data.iloc[:,1:25].copy()
use_df.columns = use_df.loc[0]
use_df = use_df.drop(use_df.index[0])
last_week_ratios, last_year_ratios, curr_week_ratios = calculate_ratios(use_df)

relevant_cols = use_df.columns
# formatting ratios
formatted_dvlw = format_ratio(last_week_ratios[0])
formatted_dvly = format_ratio(last_year_ratios[0])
formatted_uvlw = format_ratio(last_week_ratios[1])
formatted_uvly = format_ratio(last_year_ratios[1]) 
formatted_dslw = format_ratio(last_week_ratios[2])
formatted_dsly = format_ratio(last_year_ratios[2])
formatted_uslw = format_ratio(last_week_ratios[3])
formatted_usly = format_ratio(last_year_ratios[3])
formatted_fivehun_dslw = format_ratio(fivehun_dslw)
formatted_fivehun_dsly = format_ratio(fivehun_dsly)
formatted_thou_dslw = format_ratio(thou_dslw)
formatted_thou_dsly = format_ratio(thou_dsly)

dollar_share_per = curr_week_ratios[0]
unit_share_per = curr_week_ratios[1]

hisense_BPI = round((dollar_share_per[1] / unit_share_per[1]) * 100)
hisense_data = (use_df.iloc[1,int(len(relevant_cols)/2): len(relevant_cols)]).tolist()

hisense_BPI_lw = (hisense_data[2] / hisense_data[8])*100
hisense_BPI_ly = (float(hisense_data[1]) / float(hisense_data[7]))*100
hisense_BPI_wow = round((((dollar_share_per[1] / unit_share_per[1]) * 100) - hisense_BPI_lw))
hisense_BPI_yoy = round((((dollar_share_per[1] / unit_share_per[1]) * 100) - hisense_BPI_ly))
hisense_BPI_YTD = round((float(hisense_data[5]) / float(hisense_data[11]))*100)

TCL_BPI = round((dollar_share_per[5] / unit_share_per[5]) * 100)
TCL_data = (use_df.iloc[5,int(len(relevant_cols)/2): len(relevant_cols)]).tolist()
TCL_BPI_lw = (float(TCL_data[2]) / float(TCL_data[8]))*100
TCL_BPI_ly = (float(TCL_data[1]) / float(TCL_data[7]))*100
TCL_BPI_wow = round((((dollar_share_per[5] / unit_share_per[5]) * 100) - TCL_BPI_lw))
TCL_BPI_yoy = round((((dollar_share_per[5] / unit_share_per[5]) * 100) - TCL_BPI_ly))
TCL_BPI_YTD = round((float(TCL_data[5]) / float(TCL_data[11]))*100)

c_line0 = f'第{current_nweek}周行业销售额同比: {formatted_dvly[0]}%，环比: {formatted_dvlw[0]}%；行业销售量同比: {formatted_uvly[0]}%，环比: {formatted_uvlw[0]}%'
c_line1 = f'第{current_nweek}周海信销售额同比: {formatted_dvly[1]}%，环比: {formatted_dvlw[1]}%；海信销售量同比: {formatted_uvly[1]}%，环比: {formatted_uvlw[1]}%'
c_line2 = f'第{current_nweek}周三星销售额同比: {formatted_dvly[3]}%，环比: {formatted_dvlw[3]}%；三星销售量同比: {formatted_uvly[3]}%，环比: {formatted_uvlw[3]}%. TCL销售额同比: {formatted_dvly[5]}%，环比: {formatted_dvlw[5]}%；TCL销量同比: {formatted_uvly[5]}%，环比: {formatted_uvlw[5]}%. Vizio销售额同比: {formatted_dvly[6]}%，环比: {formatted_dvlw[6]}%；Vizio销量同比: {formatted_uvly[6]}%，环比: {formatted_uvlw[6]}%'
c_line3 = f'第{current_nweek}周海信额占有率: {dollar_share_per[1]}%，同比: {formatted_dsly[1]}，环比: {formatted_dslw[1]}；量占有率: {unit_share_per[1]}%，同比: {formatted_usly[1]}，环比: {formatted_uslw[1]}。'
c_line4 = f'第{current_nweek}周三星额占有率: {dollar_share_per[3]}%，同比: {formatted_dsly[3]}，环比: {formatted_dslw[3]}；量占有率: {unit_share_per[3]}%，同比: {formatted_usly[3]}，环比: {formatted_uslw[3]}。TCL额占有率: {dollar_share_per[5]}%，同比: {formatted_dsly[5]}，环比: {formatted_dslw[5]}；量占有率: {unit_share_per[5]}%，同比: {formatted_usly[5]}，环比: {formatted_uslw[5]}。VIZIO额占有率: {dollar_share_per[6]}%，同比: {formatted_dsly[6]}，环比: {formatted_dslw[6]}；量占有率: {unit_share_per[6]}%，同比: {formatted_usly[6]}，环比: {formatted_uslw[6]}。'
c_line5 = f'第{current_nweek}周海信500$+ 额占率: {fivehun_dollar_share[0]}%，同比: {formatted_fivehun_dsly[0]}，环比: {formatted_fivehun_dslw[0]}。1000$+ 额占率: {thou_dollar_share[0]}%，同比: {formatted_thou_dsly[0]}，环比: {formatted_thou_dslw[0]}。'
c_line6 = f'第{current_nweek}周三星500$+ 额占率: {fivehun_dollar_share[2]}%，同比: {formatted_fivehun_dsly[2]}，环比: {formatted_fivehun_dslw[2]}。TCL 500$+ 额占率: {fivehun_dollar_share[4]}%，同比: {formatted_fivehun_dsly[4]}，环比: {formatted_fivehun_dslw[4]}。Vizio 500$+ 额占率: {fivehun_dollar_share[5]}%，同比: {formatted_fivehun_dsly[5]}，环比: {formatted_fivehun_dslw[5]}。'
c_line7 = f'第{current_nweek}周三星1000$+ 额占率: {thou_dollar_share[2]}%，同比: {formatted_thou_dsly[2]}，环比: {formatted_thou_dslw[2]}。TCL 1000$+ 额占率: {thou_dollar_share[4]}%，同比: {formatted_thou_dsly[4]}，环比: {formatted_thou_dslw[4]}。'

e_line0 = f'At Week {current_nweek}, Industry dollar sales YOY: {formatted_dvly[0]}%, WOW: {formatted_dvlw[0]}%; Industry unit sales YOY: {formatted_uvly[0]}%, WOW: {formatted_uvlw[0]}%.'
e_line1 = f'At Week {current_nweek}, Hisense dollar sales YOY: {formatted_dvly[1]}%, WOW: {formatted_dvlw[1]}%; Hisense unit sales YOY: {formatted_uvly[1]}%, WOW: {formatted_uvlw[1]}%.'
e_line2 = f'At Week {current_nweek}, Samsung dollar sales YOY: {formatted_dvly[3]}%, WOW: {formatted_dvlw[3]}%; Samsung unit sales YOY: {formatted_uvly[3]}%, WOW: {formatted_uvlw[3]}%. TCL dollar sales YOY: {formatted_dvly[5]}%, WOW: {formatted_dvlw[5]}%; TCL unit sales YOY: {formatted_uvly[5]}%, WOW: {formatted_uvlw[5]}%. Vizio dollar sales YOY: {formatted_dvly[6]}%, WOW: {formatted_dvlw[6]}%; Vizio unit sales YOY: {formatted_uvly[6]}%, WOW: {formatted_uvlw[6]}%.'
e_line3 = f'At Week {current_nweek}, Hisense dollar share: {dollar_share_per[1]}%, YoY: {formatted_dsly[1]}, WoW: {formatted_dslw[1]}; Unit Share: {unit_share_per[1]}%, YoY: {formatted_usly[1]}, WoW: {formatted_uslw[1]}.'
e_line4 = f'At Week {current_nweek}, Samsung dollar share: {dollar_share_per[3]}%, YoY: {formatted_dsly[3]}, WoW: {formatted_dslw[3]}; Unit Share: {unit_share_per[3]}%, YoY: {formatted_usly[3]}, WoW: {formatted_uslw[3]}. TCL dollar share: {dollar_share_per[5]}%, YoY: {formatted_dsly[5]}, WoW: {formatted_dslw[5]}; Unit Share: {unit_share_per[5]}%, YoY: {formatted_usly[5]}, WoW: {formatted_uslw[5]}. Vizio dollar share: {dollar_share_per[6]}%, YoY: {formatted_dsly[6]}%, WoW: {formatted_dslw[6]}; Unit Share: {unit_share_per[6]}%，YoY: {formatted_usly[6]}，WoW: {formatted_uslw[6]}.'
e_line5 = f'At Week {current_nweek}, Hisense 500$+ Dollar share: {fivehun_dollar_share[0]}%, YoY: {formatted_fivehun_dsly[0]}, WoW: {formatted_fivehun_dslw[0]}. 1000$+ Dollar share: {thou_dollar_share[0]}%，YoY: {formatted_thou_dsly[0]}，WoW: {formatted_thou_dslw[0]}。'
e_line6 = f'At Week {current_nweek}, Samsung 500$+ Dollar Share: {fivehun_dollar_share[2]}%, YoY: {formatted_fivehun_dsly[2]}, WoW: {formatted_fivehun_dslw[2]}. TCL 500$+ Dollar Share: {fivehun_dollar_share[4]}%, YoY: {formatted_fivehun_dsly[4]}, WoW: {formatted_fivehun_dslw[4]}. Vizio 500$+ Dollar Share: {fivehun_dollar_share[5]}%, YoY: {formatted_fivehun_dsly[5]}, WoW: {formatted_fivehun_dslw[5]}.'
e_line7 = f'At Week {current_nweek}, Samsung 1000$+ Dollar share: {thou_dollar_share[2]}%，YoY: {formatted_thou_dsly[2]}，WoW: {formatted_thou_dslw[2]}。TCL 1000$+ Dollar share: {thou_dollar_share[4]}%，YoY: {formatted_thou_dsly[4]}，WoW: {formatted_thou_dslw[4]}。'

final_sheet = pd.DataFrame({'第二部分：市场行业情况                           2nd part: Industry and market analysis report':[c_line0, c_line1, c_line2, c_line3, c_line4, c_line5, c_line6, c_line7
                                                                                                                   , e_line0, e_line1, e_line2,e_line3, e_line4, e_line5, e_line6, e_line7]})
# final_sheet = pd.DataFrame({'第二部分：市场行业情况                           2nd part: Industry and market analysis report':[c_line0, c_line1, c_line2, c_line3, c_line4, e_line0, e_line1, e_line2,e_line3, e_line4]})

final_sheet

,第二部分：市场行业情况 2nd part: Industry and market analysis report
0,第6周行业销售额同比: -4.1%，环比: +18.0%；行业销售量同比: -0.6%，环比...
1,第6周海信销售额同比: -12.1%，环比: +8.3%；海信销售量同比: -30.0%，环...
2,第6周三星销售额同比: +6.1%，环比: +30.8%；三星销售量同比: +16.0%，环...
3,第6周海信额占有率: 7.1%，同比: -0.6，环比: -0.6；量占有率: 9.6%，同...
4,第6周三星额占有率: 38.2%，同比: +3.7，环比: +3.8；量占有率: 24.1%...
5,第6周海信500$+ 额占率: 3.3%，同比: +1.1，环比: -0.7。1000$+ ...
6,第6周三星500$+ 额占率: 48.9%，同比: +5.3，环比: +2.2。TCL 50...
7,第6周三星1000$+ 额占率: 48.1%，同比: -0.9，环比: +2.4。TCL 1...
8,"At Week 6, Industry dollar sales YOY: -4.1%, W..."
9,"At Week 6, Hisense dollar sales YOY: -12.1%, W..."


In [4]:
BPI_file = pd.read_excel('weekly market changes, BPI.xlsx')
BPI_file.columns = BPI_file.loc[2]
BPI_file = BPI_file.drop(BPI_file.index[0:3])
tv_sizes = ['50 inches', '55 inches', '65 inches', '75 inches']
BPI_brands = ['Hisense', 'TCL']
annual_target = ['']

BPI_data = BPI_file[BPI_file['Display Size'].isin(tv_sizes)]
BPI_data = BPI_data[BPI_data['Brand'].isin(BPI_brands)]
BPI_data = BPI_data.sort_values(['Brand', 'Display Size'])

hss_tv_sizes = tv_sizes.insert(0, '海信整体') 
tcl_tv_sizes = tv_sizes.insert(0, 'TCL整体') 

BPI_data['BPI_this_week'] = (BPI_data.iloc[:, 5] / BPI_data.iloc[:, 11] * 100).astype('float')
BPI_data['WoW'] = (BPI_data['BPI_this_week'] - (BPI_data.iloc[:, 4] / BPI_data.iloc[:, 10] * 100).astype('float'))
BPI_data['YoY'] = (BPI_data['BPI_this_week'] - (BPI_data.iloc[:, 3] / BPI_data.iloc[:, 9] * 100).astype('float'))
BPI_data['YTD_BPI'] = (BPI_data.iloc[:, 7] / BPI_data.iloc[:, 13] * 100).astype('float')

BPI_data.BPI_this_week = BPI_data.BPI_this_week.round().astype('int')
BPI_data.WoW = BPI_data.WoW.round().astype('int')
BPI_data.YoY = BPI_data.YoY.round().astype('int')
BPI_data.YTD_BPI = BPI_data.YTD_BPI.round().astype('int')

BPI_data = BPI_data.reset_index()
BPI_data = BPI_data.drop('index', axis = 1)

BPI_test = BPI_data.copy()
BPI_test = BPI_test.drop(BPI_test.columns[2:13], axis=1)
BPI_test = BPI_test.drop('Brand', axis=1)
BPI_test = BPI_test.dropna(axis=1,how = 'all')

midpoint = len(BPI_test)//2
u_half = BPI_test.iloc[:midpoint]
l_half = BPI_test.iloc[midpoint:]
l_half = l_half.reset_index()
l_half = l_half.drop('index', axis = 1)

first_row = 0
concat_df = pd.concat([u_half, l_half], axis=1)
concat_df.loc[-1] = ['海信整体',hisense_BPI, hisense_BPI_wow,hisense_BPI_yoy,hisense_BPI_YTD,'TCL整体',TCL_BPI, TCL_BPI_wow,TCL_BPI_yoy,TCL_BPI_YTD]
concat_df.index = concat_df.index + 1
concat_df = concat_df.sort_index() 
Annual_Target = [67,90,75,64,69]
concat_df.insert(1, 'Annual Target', Annual_Target)
concat_df

2,Display Size,Annual Target,BPI_this_week,WoW,YoY,YTD_BPI,Display Size,BPI_this_week,WoW,YoY,YTD_BPI
0,海信整体,67,72,-1,4,67,TCL整体,74,-1,14,69
1,50 inches,90,84,-2,-4,90,50 inches,85,-2,-7,90
2,55 inches,75,79,2,14,72,55 inches,67,1,-3,64
3,65 inches,64,71,-4,13,66,65 inches,72,-4,10,65
4,75 inches,69,71,0,1,67,75 inches,70,-1,-2,67


In [33]:
date = datetime.today().strftime('%Y-%m-%d')
final_sheet.to_excel(f'Part II Chart {date} .xlsx',encoding = 'utf_8_sig')
concat_df.to_excel(f'Part VI Chart {date}.xlsx',encoding = 'utf_8_sig')